This notebook introduce more advanced techniques in Forecasting tasks.

Similar to the quick start tutourial, we will do forecasting related to the COV19 dataset.

In [ ]:
from autogluon.forecasting import Forecasting as task
from autogluon.forecasting import ForecastingPredictor

train_data = task.Dataset("https://autogluon.s3-us-west-2.amazonaws.com/datasets/CovidTimeSeries/train.csv")
test_data = task.Dataset("https://autogluon.s3-us-west-2.amazonaws.com/datasets/CovidTimeSeries/test.csv")

# uncessary as it's the default metric
eval_metric = "mean_wQuantileLoss"
# uncessary as it's the default searcher
searcher_type = "random"

# Specify hyperparameters and tuning them

We can do hyperparameter tuning using autogluon, and here the variable "context_length" is the one that we would like to tune.

In [ ]:
import autogluon.core as ag
from gluonts.distribution.neg_binomial import NegativeBinomialOutput


mqcnn_params = {
    "context_length": ag.Int(75, 100),
    "num_batches_per_epoch": 10,
    "epochs": 30
}

deepar_params = {
    "context_length": ag.Int(75, 100),
    "num_batches_per_epoch": 10,
    "distr_output": NegativeBinomialOutput(),
    "epochs": 30
}

sff_params = {
    "context_length": ag.Int(75, 100),
    "num_batches_per_epoch": 30,
    "epochs": 30
}

predictor = task.fit(train_data=train_data,
                     prediction_length=19,
                     index_column="name",
                     target_column="ConfirmedCases",
                     time_column="Date",
                     hyperparameter_tune=True,
                     quantiles=[0.1, 0.5, 0.9],
                     hyperparameters={"MQCNN": mqcnn_params,
                                      "DeepAR": deepar_params,
                                      "SFF": sff_params
                                      },
                     search_strategy=searcher_type,
                     eval_metric=eval_metric,
                     num_trials=3, )

In [ ]:
predictor.fit_summary()

# Evaluation and Predictions

We again demonstrate how to use the trained models to predict on the test data.

To see the performance of each model on test data, we can use the leaderboard() method.

In [ ]:
predictor.leaderboard(test_data)

By default, the predictor will use the best more on the validation set to do the prediction. The prediction results we get is a dictionary whose key is each time series's index and value is the corresponding prediction dataframe.

In [ ]:
predictions = predictor.predict(test_data)
predictions["Afghanistan_"]

To see which model is the best, we can call the get_model_best() method for trainer.

In [ ]:
predictor._trainer.get_model_best()

Besides using the best model on validation data, we can also specify which model we want to use for predictions.

In [ ]:
model_trained = predictor._trainer.get_model_names_all()
specific_model = predictor._trainer.load_model(model_trained[0])
specific_model.get_info()

In [ ]:
specific_predictions = predictor.predict(test_data, model=specific_model)
specific_predictions["Afghanistan_"]